<a href="https://colab.research.google.com/github/PhilBurub/ML_course_MSc/blob/main/HW3_recsys/HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ДЗ №3 Двухуровневый пайплайн
#### В этой домашке вам предстоит написать с нуля двустадийную рекомендательную систему.

#### Дата выдачи: 10.03.25

#### Мягкий дедлайн: 31.03.25 23:59 MSK

#### Жесткий дедлайн: 7.04.25 23:59 MSK

In [ ]:
!pip install numpy==1.23.2

In [ ]:
%%capture
!pip install rectools[torch]==0.12.0
!pip install implicit
!export OPENBLAS_NUM_THREADS=1

### Описание
Это творческое задание, в котором вам необходимо реализовать полный цикл построения рекомендательной системы: реализовать кандидат генераторов, придумать и собрать признаки, обучить итоговый ранкер и заинференсить модели на всех пользователей.

Вам предоставляется два набора данных: `train.csv` и `test.csv`

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive




### 1 Этап. Модели первого уровня. (max 3 балла)
В этом этапе вам необходимо разделить `train` датасет на 2 части: для обучения моделей первого уровня и для их валидации. Единственное условие для разбиения – разбивать нужно по времени. Данные для обучение будем называть `train_stage_1`, данные для валидации `valid_stage_1`. Объемы этих датасетов вы определяет самостоятельно.

Для начала нам нужно отобрать кандидатов при помощи легких моделей. Необходимо реализовать 3 типа моделей:
1. Любая эвристическая(алгоритмичная) модель на ваш выбор **(0.5 балл)**
2. Любая матричная факторизация на ваш выбор **(1 балл)**
3. Любая нейросетевая модель на ваш выбор **(1 балла)**

Не забудьте использовать скор каждой модели, как признак!



In [ ]:
import pandas as pd
import numpy as np
from datetime import date
from matplotlib import pyplot as plt
from sklearn.metrics import ndcg_score

#### 1. Разделение данных

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/AI/recsys/train_part.csv')
train.head()

,user_id,item_id,last_watch_dt,total_dur,watched_pct,target
0,310745,13373,2021-03-13,4485,98.0,1
1,952323,15997,2021-03-13,7507,100.0,1
2,889459,11460,2021-03-13,60,0.0,0
3,854016,11237,2021-03-13,5381,98.0,1
4,307257,9132,2021-03-13,5814,100.0,1


In [ ]:
train.head()

,user_id,item_id,last_watch_dt,total_dur,watched_pct,target
0,310745,13373,2021-03-13,4485,98.0,1
1,952323,15997,2021-03-13,7507,100.0,1
2,889459,11460,2021-03-13,60,0.0,0
3,854016,11237,2021-03-13,5381,98.0,1
4,307257,9132,2021-03-13,5814,100.0,1


In [ ]:
train.last_watch_dt = train.last_watch_dt.apply(date.fromisoformat)

In [ ]:
threshold = train.last_watch_dt.quantile(q=0.75, interpolation='nearest')
train.last_watch_dt.min(), threshold, train.last_watch_dt.max()

(datetime.date(2021, 3, 13),
 datetime.date(2021, 7, 21),
 datetime.date(2021, 8, 12))

In [ ]:
train_stage_1, valid_stage_1 = train[train.last_watch_dt <= threshold], train[train.last_watch_dt > threshold]

#### 2. Эвристическая модель
Будем за скор считать количество интеракций, причем будем брать сглаживание по месяцам и target=0 считать с отрицательным весом

In [ ]:
class TopPopular:
  def fit(self, df):
    df = df.copy()
    last = df.last_watch_dt.max()
    coef = df.last_watch_dt.apply(lambda x: (last - x).days // 30 + 1)
    df['value'] = df.target.map({1: 1, 0: -1}) / coef
    self.preds = df.groupby('item_id').agg({'value': 'sum'})\
      .sort_values(by='value', ascending=False)

  def score(self, item_id):
    return self.preds.loc[item_id].value

  def top_n(self, n):
    return self.preds.index[:n]

In [ ]:
heuristic = TopPopular()
heuristic.fit(train_stage_1)
heuristic.top_n(10)

Index([13865, 15297, 3734, 9728, 142, 8636, 11237, 7417, 1844, 14431], dtype='int64', name='item_id')

#### 3. Модель матричного разложения
Возьму iALS

In [ ]:
from implicit.cpu.als import AlternatingLeastSquares
from scipy.sparse import csr_matrix

In [ ]:
shape = (
    train_stage_1.user_id.max() + 1,
    train_stage_1.item_id.max() + 1
)
smoothing_coef = train_stage_1.last_watch_dt.apply(
    lambda x: (threshold - x).days // 30 + 1
)

user_item_matrix = csr_matrix(
    (
        train_stage_1.target.map({1: 1, 0: -1}) / smoothing_coef,
        train_stage_1[['user_id', 'item_id']].T.values
    ),
    shape=shape
)

In [ ]:
ials = AlternatingLeastSquares(factors=15, calculate_training_loss=True)
ials.fit(user_item_matrix)

/usr/local/lib/python3.11/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 2 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
ials.save('/content/drive/MyDrive/AI/recsys/ials_model')

In [ ]:
ials.recommend(5, user_item_matrix[5])

(array([ 4495,  9728,  3734, 16166,  3182,  7626, 16270, 12173,  5411,
        10440], dtype=int32),
 array([3.8692617e-12, 3.7411359e-12, 3.6468276e-12, 1.0599452e-12,
        8.7241092e-13, 6.5465938e-13, 5.5338904e-13, 5.4478888e-13,
        4.8884377e-13, 4.8013091e-13], dtype=float32))

#### 4. Нейросетевая модель
Возьму BERT4Rec

In [ ]:
from rectools.models.nn.transformers.bert4rec import BERT4RecModel
from rectools.dataset import Dataset

In [ ]:
%%capture
interactions_df = train_stage_1[['user_id', 'item_id']]
interactions_df['datetime'] = train_stage_1['last_watch_dt']
interactions_df['weight'] = train_stage_1.target
dataset = Dataset.construct(interactions_df)

In [ ]:
bert4rec = BERT4RecModel(n_heads=2, n_factors=64, epochs=1, batch_size=256)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
bert4rec.fit(dataset)

/usr/local/lib/python3.11/dist-packages/rectools/dataset/identifiers.py:60: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  unq_values = pd.unique(values)
/usr/local/lib/python3.11/dist-packages/rectools/models/nn/item_net.py:129: UserWarning: Ignoring `CatFeaturesItemNet` block because dataset doesn't contain item features.
  warnings.warn(explanation)
/usr/local/lib/python3.11/dist-packages/pydantic/main.py:453: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [input_value=('rectools.models.nn.item...net.CatFeaturesItemNet'), input_type=tuple])
  return self.__pydantic_serializer__.to_python(
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
INFO:pytorch_lightning.acce

In [ ]:
bert4rec.save('/content/drive/MyDrive/AI/recsys/nn_model')

12476053

#### 5. Объединение и анализ

In [ ]:
my_heuristic_model = heuristic
my_matrix_factorization = AlternatingLeastSquares.load('/content/drive/MyDrive/AI/recsys/ials_model')
my_neural_network = BERT4RecModel.load('/content/drive/MyDrive/AI/recsys/nn_model')

/usr/local/lib/python3.11/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 2 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/dist-packages/rectools/models/nn/item_net.py:129: UserWarning: Ignoring `CatFeaturesItemNet` block because dataset doesn't contain item features.
  warnings.warn(explanation)


In [ ]:
top_n = 10
val_users = valid_stage_1.user_id.unique()
train_users = train_stage_1.user_id.value_counts()

warmer_users = val_users[
    np.in1d(
        val_users,
        train_users[train_users>1].index
    )
]

warm_users = val_users[
    np.in1d(
        val_users,
        train_users.index
    )
]

cold_users = val_users[
      ~np.in1d(
        val_users,
        train_users.index
    )
]

In [ ]:
warmer_users_df = pd.DataFrame(index=warmer_users)

bert_preds = my_neural_network.recommend(warmer_users, dataset, top_n, filter_viewed=True)
bert_preds_grouped = bert_preds.groupby('user_id').agg({'item_id': list}).loc[warmer_users]
warmer_users_df['nn_items'] = bert_preds_grouped.item_id

In [ ]:
warm_users_df = pd.DataFrame(index=warm_users)

mf_items, mf_scores = my_matrix_factorization.recommend(warm_users, user_item_matrix[warm_users], N=top_n)
warm_users_df['mf_items'] = mf_items.tolist()

In [ ]:
val_users_df = pd.DataFrame(index=val_users)

topp_preds = my_heuristic_model.preds[:top_n]
val_users_df['pop_items'] = [topp_preds.index.tolist()] * len(val_users)

In [ ]:
light_recs = pd.concat(
    (
        val_users_df,
        warm_users_df,
        warmer_users_df
    ),
    axis=1
)

In [ ]:
light_recs.to_json('/content/drive/MyDrive/AI/recsys/light_models.recs')

Каждая модель должна уметь:
1) для пары user_item предсказывать скор релевантности (масштаб скора не важен), важно обработать случаи, когда модель не можеn проскорить пользователя или айтем, вместо этого вернуть какое-то дефолтное значение
2) для всех пользователей вернуть top-k самых релевантных айтемов (тут вам скоры не нужны)


Дополнительно можно провести анализ кандидат генератов, измерить насколько различные айтемы они рекомендуют, например с помощью таких метрик как: [Ranked based overlap](https://github.com/changyaochen/rbo) или различные вариации [Diversity](https://github.com/MaurizioFD/RecSys2019_DeepLearning_Evaluation/blob/master/Base/Evaluation/metrics.py#L289). **(1 балл)**

In [ ]:
!pip install rbo -q

In [ ]:
import rbo

light_recs = pd.read_json('/content/drive/MyDrive/AI/recsys/light_models.recs')

In [ ]:
def get_score(a, b):
  return rbo.RankingSimilarity(a, b).rbo()

In [ ]:
light_recs.head(3)

,pop_items,mf_items,nn_items
363436,"[13865, 15297, 3734, 9728, 142, 8636, 11237, 7...","[12995, 7626, 12173, 4457, 16166, 10942, 3182,...","[9728, 10440, 3734, 15297, 8636, 11237, 1844, ..."
1055286,"[13865, 15297, 3734, 9728, 142, 8636, 11237, 7...","[7102, 1844, 4457, 7626, 12173, 12995, 7417, 4...","[15297, 10440, 13865, 3734, 142, 1844, 8636, 6..."
328281,"[13865, 15297, 3734, 9728, 142, 8636, 11237, 7...","[4495, 7626, 16166, 12173, 3182, 13865, 4151, ...","[9728, 13865, 10440, 3734, 11237, 15297, 8636,..."


In [ ]:
topp_mf_scores = []
topp_nn_scores = []
nn_mf_scores = []

for _, row in light_recs.iterrows():
  if row.pop_items is not None and row.mf_items is not None:
    topp_mf_scores.append(get_score(row.pop_items, row.mf_items))
  if row.pop_items is not None and row.nn_items is not None:
      topp_nn_scores.append(get_score(row.pop_items, row.nn_items))
  if row.mf_items is not None and row.nn_items is not None:
      nn_mf_scores.append(get_score(row.nn_items, row.mf_items))

In [ ]:
pd.DataFrame(
    [
        {'approaches': 'heuristic x mf', 'mean_score': np.mean(topp_mf_scores), 'median_score': np.median(topp_mf_scores)},
        {'approaches': 'heuristic x nn', 'mean_score': np.mean(topp_nn_scores), 'median_score': np.median(topp_nn_scores)},
        {'approaches': 'nn x mf', 'mean_score': np.mean(nn_mf_scores), 'median_score': np.median(nn_mf_scores)}
    ]
)

,approaches,mean_score,median_score
0,heuristic x mf,0.083076,0.047897
1,heuristic x nn,0.518648,0.557976
2,nn x mf,0.117060,0.057897


Что интересно, BERT и toppopular показали самые близкие результаты


### 2 Этап. Генерация и сборка признаков. (max 2 балла)
Необходимо собрать минимум 10 осмысленных (`np.radndom.rand()` не подойдет) признаков, при этом:
1. 2 должны относиться только к сущности "пользователь" (например средний % просмотра фильмов у этой возрастной категории)
2. 2 должны относиться только к сущности "айтем" (например средний средний % просмотра данного фильма)
3. 6 признаков, которые показывают связь пользователя и айтема (например средний % просмотра фильмов с данным актером (айтем) у пользователей с таким же полом (пользователь)).

### ВАЖНО!  

1. **В датасете есть колонка `watched_prct`. Ее можно использовать для генерации признаков (например сколько пользователь в среднем смотрит фильмы), но нельзя подавать в модель, как отдельную фичу, потому что она напрямую связана с target.**
2. **Все признаки должны быть собраны без дата лика, то есть если пользователь посмотрел фильм 10 августа, то признаки мы можем считать только на данных до 9 августа включительно.**


### Разбалловка
Обучение ранкера будет проходить на `valid_stage_1`, как  раз на которой мы валидировали модели, а тестировать на `test`. Поэтому есть 2 варианта сборки признаков, **реализовать нужно только 1 из них:**
1. Для обучения собираем признаки на первый день `valid_stage_1`, а для теста на первый день `test`. Например, если `valid_stage_1` начинается 5 сентября, то все признаки мы можем собирать только по 4 сентября включительно. **(1 балл)**
2. Признаки будем собирать честно на каждый день, то есть на 5 сентября собираем с начала до 4, на 6 сентября с начала до 5 и т.д. **(2 балла)**

In [ ]:
users = pd.read_csv('/content/drive/MyDrive/AI/recsys/users.csv')
items = pd.read_csv('/content/drive/MyDrive/AI/recsys/items.csv')[
    ['item_id', 'content_type', 'release_year', 'genres', 'countries', 'age_rating', 'directors', 'actors']
]

#### train

In [ ]:
train_feartures_users = train_stage_1\
  .groupby('user_id')\
  .agg({'watched_pct': 'mean'})\
  .reset_index()\
  .merge(users, 'left', on='user_id')

age = train_feartures_users.groupby('age').agg({'watched_pct': 'mean'}).watched_pct.to_dict()
income = train_feartures_users.groupby('income').agg({'watched_pct': 'mean'}).watched_pct.to_dict()

In [ ]:
train_feartures_items = train_stage_1\
  .groupby('item_id')\
  .agg({'watched_pct': 'mean'})\
  .reset_index()\
  .merge(items, 'left', on='item_id')

content_type = train_feartures_items.groupby('content_type').agg({'watched_pct': 'mean'}).watched_pct.to_dict()
age_rating = train_feartures_items.groupby('age_rating').agg({'watched_pct': 'mean'}).watched_pct.to_dict()

In [ ]:
train_feartures_combined = train_stage_1\
  .merge(items, 'left', on='item_id')\
  .merge(users, 'left', on='user_id')

train_feartures_combined['decade'] = train_feartures_combined.release_year // 10
train_feartures_combined['director'] = train_feartures_combined.directors.apply(lambda x: x.split(', ')[0] if isinstance(x, str) else x)
train_feartures_combined['country'] = train_feartures_combined.countries.apply(lambda x: x.split(', ')[0] if isinstance(x, str) else x)
train_feartures_combined['genre'] = train_feartures_combined.genres.apply(lambda x: x.split(', ')[0] if isinstance(x, str) else x)
train_feartures_combined['actor'] = train_feartures_combined.actors.apply(lambda x: x.split(', ')[0] if isinstance(x, str) else x)

age_decade = train_feartures_combined.groupby(['age', 'decade']).agg({'watched_pct': 'mean'}).watched_pct.to_dict()
kids_age_rating = train_feartures_combined.groupby(['kids_flg', 'age_rating']).agg({'watched_pct': 'mean'}).watched_pct.to_dict()
income_country = train_feartures_combined.groupby(['income', 'country']).agg({'watched_pct': 'mean'}).watched_pct.to_dict()
sex_director = train_feartures_combined.groupby(['sex', 'director']).agg({'watched_pct': 'mean'}).watched_pct.to_dict()
age_genre = train_feartures_combined.groupby(['age', 'genre']).agg({'watched_pct': 'mean'}).watched_pct.to_dict()
sex_actor = train_feartures_combined.groupby(['sex', 'actor']).agg({'watched_pct': 'mean'}).watched_pct.to_dict()

In [ ]:
val_feartures_combined = valid_stage_1\
  .merge(items, 'left', on='item_id')\
  .merge(users, 'left', on='user_id')

In [ ]:
%%capture
valid_stage_1['age'] = val_feartures_combined.age.map(age).values
valid_stage_1['income'] = val_feartures_combined.income.map(income).values
valid_stage_1['content_type'] = val_feartures_combined.content_type.map(content_type).values
valid_stage_1['age_rating'] = val_feartures_combined.age_rating.map(age_rating).values

valid_stage_1['age_decade'] = val_feartures_combined.apply(
    lambda x: age_decade.get((x.age, x.release_year // 10)),
    axis=1
).values
valid_stage_1['kids_age_rating'] = val_feartures_combined.apply(
    lambda x: kids_age_rating.get((x.kids_flg, x.age_rating)),
    axis=1
).values
valid_stage_1['income_country'] = val_feartures_combined.apply(
    lambda x: income_country.get((x.income, (x.countries if isinstance(x.countries, str) else '').split(', ')[0])),
    axis=1
).values
valid_stage_1['sex_director'] = val_feartures_combined.apply(
    lambda x: sex_director.get((x.sex, (x.directors if isinstance(x.directors, str) else '').split(', ')[0])),
    axis=1
).values
valid_stage_1['age_genre'] = val_feartures_combined.apply(
    lambda x: age_genre.get((x.age, (x.genres if isinstance(x.genres, str) else '').split(', ')[0])),
    axis=1
).values
valid_stage_1['sex_actor'] = val_feartures_combined.apply(
    lambda x: sex_actor.get((x.sex, (x.actors if isinstance(x.actors, str) else '').split(', ')[0])),
    axis=1
).values

In [ ]:
def get_mf_score(row):
    if row.user_id not in warm_users:
      return 0
    items, scores = my_matrix_factorization.recommend(
          row.user_id,
          user_item_matrix[row.user_id],
          N=user_item_matrix.shape[1]
    )
    if row.item_id not in items:
      return 0
    item_score_idx = np.where(items == row.item_id)[0].item()
    return scores[item_score_idx]

def get_nn_score(row):
    if row.user_id not in warmer_users:
      return 0
    df = my_neural_network.recommend(
          [row.user_id],
          dataset,
          k=len(dataset.item_id_map.external_ids),
          filter_viewed=False
    )

    item_row = df[df.item_id == row.item_id]
    if len(item_row) != 1:
      return 0
    return item_row.score.item()

In [ ]:
nn_recs = my_neural_network.recommend(
      warmer_users,
      dataset,
      k=len(dataset.item_id_map.external_ids),
      filter_viewed=False
)

In [ ]:
valid_stage_1['heuristic_score'] = valid_stage_1.item_id.apply(lambda x: heuristic.score(x) if x in heuristic.preds.index else 0)
valid_stage_1['mf_score'] = valid_stage_1.apply(get_mf_score, axis=1)
valid_stage_1['nn_score'] = valid_stage_1.apply(get_nn_score, axis=1)

In [ ]:
valid_stage_1.to_json('/content/drive/MyDrive/AI/recsys/train_df_with_features.recs')

#### test

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/AI/recsys/test_part.csv')

In [ ]:
test_feartures_users = train\
  .groupby('user_id')\
  .agg({'watched_pct': 'mean'})\
  .reset_index()\
  .merge(users, 'left', on='user_id')

age = test_feartures_users.groupby('age').agg({'watched_pct': 'mean'}).watched_pct.to_dict()
income = test_feartures_users.groupby('income').agg({'watched_pct': 'mean'}).watched_pct.to_dict()

In [ ]:
test_feartures_items = train\
  .groupby('item_id')\
  .agg({'watched_pct': 'mean'})\
  .reset_index()\
  .merge(items, 'left', on='item_id')

content_type = test_feartures_items.groupby('content_type').agg({'watched_pct': 'mean'}).watched_pct.to_dict()
age_rating = test_feartures_items.groupby('age_rating').agg({'watched_pct': 'mean'}).watched_pct.to_dict()

In [ ]:
test_feartures_combined = train\
  .merge(items, 'left', on='item_id')\
  .merge(users, 'left', on='user_id')

test_feartures_combined['decade'] = test_feartures_combined.release_year // 10
test_feartures_combined['director'] = test_feartures_combined.directors.apply(lambda x: x.split(', ')[0] if isinstance(x, str) else x)
test_feartures_combined['country'] = test_feartures_combined.countries.apply(lambda x: x.split(', ')[0] if isinstance(x, str) else x)
test_feartures_combined['genre'] = test_feartures_combined.genres.apply(lambda x: x.split(', ')[0] if isinstance(x, str) else x)
test_feartures_combined['actor'] = test_feartures_combined.actors.apply(lambda x: x.split(', ')[0] if isinstance(x, str) else x)

age_decade = test_feartures_combined.groupby(['age', 'decade']).agg({'watched_pct': 'mean'}).watched_pct.to_dict()
kids_age_rating = test_feartures_combined.groupby(['kids_flg', 'age_rating']).agg({'watched_pct': 'mean'}).watched_pct.to_dict()
income_country = test_feartures_combined.groupby(['income', 'country']).agg({'watched_pct': 'mean'}).watched_pct.to_dict()
sex_director = test_feartures_combined.groupby(['sex', 'director']).agg({'watched_pct': 'mean'}).watched_pct.to_dict()
age_genre = test_feartures_combined.groupby(['age', 'genre']).agg({'watched_pct': 'mean'}).watched_pct.to_dict()
sex_actor = test_feartures_combined.groupby(['sex', 'actor']).agg({'watched_pct': 'mean'}).watched_pct.to_dict()

In [ ]:
train_feartures_combined = train\
  .merge(items, 'left', on='item_id')\
  .merge(users, 'left', on='user_id')

In [ ]:
%%capture
test['age'] = train_feartures_combined.age.map(age).values
test['income'] = train_feartures_combined.income.map(income).values
test['content_type'] = train_feartures_combined.content_type.map(content_type).values
test['age_rating'] = train_feartures_combined.age_rating.map(age_rating).values

test['age_decade'] = train_feartures_combined.apply(
    lambda x: age_decade.get((x.age, x.release_year // 10)),
    axis=1
).values
test['kids_age_rating'] = train_feartures_combined.apply(
    lambda x: kids_age_rating.get((x.kids_flg, x.age_rating)),
    axis=1
).values
test['income_country'] = train_feartures_combined.apply(
    lambda x: income_country.get((x.income, (x.countries if isinstance(x.countries, str) else '').split(', ')[0])),
    axis=1
).values
test['sex_director'] = train_feartures_combined.apply(
    lambda x: sex_director.get((x.sex, (x.directors if isinstance(x.directors, str) else '').split(', ')[0])),
    axis=1
).values
test['age_genre'] = train_feartures_combined.apply(
    lambda x: age_genre.get((x.age, (x.genres if isinstance(x.genres, str) else '').split(', ')[0])),
    axis=1
).values
test['sex_actor'] = train_feartures_combined.apply(
    lambda x: sex_actor.get((x.sex, (x.actors if isinstance(x.actors, str) else '').split(', ')[0])),
    axis=1
).values

In [ ]:
test['heuristic_score'] = test.item_id.apply(lambda x: heuristic.score(x) if x in heuristic.preds.index else 0)
test['mf_score'] = test.apply(get_mf_score, axis=1)
test['nn_score'] = test.apply(get_nn_score, axis=1)

In [ ]:
test.to_json('/content/drive/MyDrive/AI/recsys/test_df_with_features.recs')

#### load dfs

In [ ]:
train_df_with_features = pd.read_json('/content/drive/MyDrive/AI/recsys/train_df_with_features.recs')
test_df_with_features = pd.read_json('/content/drive/MyDrive/AI/recsys/test_df_with_features.recs')


### 3 Этап. Обучение финального ранкера (max 2 балла)
Собрав все признаки из этапа 2, добавив скоры моделей из этапа 1 для каждой пары пользователь-айтем (где это возможно), пришло время обучать ранкер. В качестве ранкера можно использовать либо [xgboost](https://xgboost.readthedocs.io/en/stable/) или [catboost](https://catboost.ai/). Обучать можно как `Classfier`, так и `Ranker`, выбираем то, что лучше сработает. Обучение ранкера будет проходить на `valid_stage_1`, как  раз на которой мы валидировали модели, а тестировать на `test`, которую мы до сих пор не трогали.  Заметьте, что у нас в тесте есть холодные пользователи – те, кого не было в train и активные – те, кто был в train. Возможно их стоит обработать по отдельности (а может и нет).  
(1 балл)

После получения лучшей модели надо посмотреть на важность признаков и [shap values](https://shap.readthedocs.io/en/latest/index.html), чтобы:
1. Интерпритировать признаки, которые вы собрали, насколько они полезные
2. Проверить наличие ликов – если важность фичи в 100 раз больше, чем у всех остальных, то явно что-то не то  

(1 балл)






In [ ]:
!pip install catboost -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 50.1 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostClassifier

model = CatBoostClassifier()

In [ ]:
model.fit(train_df_with_features.drop(columns='target'), train_df_with_features.target)
pred = model.predict(test_df_with_features.drop(columns='target'))


### 4 Этап. Инференс лучшего ранкера (max 3 балла)

Теперь мы хотим построить рекомендации "на завтра", для этого нам нужно:

1. Обучить модели первого уровня на всех (train+test) данных (0.5 балла)
2. Для каждой модели первого уровня для каждого пользователя сгененировать N кандидатов (0.5 балла)
3. "Склеить" всех кандидатов для каждого пользователя (дубли выкинуть), посчитать скоры от всех моделей (0.5 балла)
4. Собрать фичи для ваших кандидатов (теперь можем считать признаки на всех данных) (0.5 балла)
5. Проскорить всех кандидатов бустингом и оставить k лучших (0.5 балла)
6. Посчитать разнообразие(Diversity) и построить график от Diversity(k) (0.5 балла)


Все гиперпараметры (N, k) определяете только Вы!

In [ ]:
# YOUR CODE HERE